# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
import warnings

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SequentialFeatureSelector

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
vehicles = pd.read_csv('data/vehicles.csv')

In [ ]:
vehicles.info()

In [ ]:
vehicles.sample(5)

In [ ]:
vehicles = vehicles.convert_dtypes()
vehicles.info()
original_row_count = vehicles.shape[0]

In [ ]:
# CALC: % of null values
vehicles.isnull().sum()/vehicles.shape[0]*100

In [ ]:
# remove a few features (columns) that are not relavent to the analysis
vehicles.drop(columns = ['id','region','VIN','state'], axis=1, inplace = True)
vehicles.columns

In [ ]:
# before dropping NaN's
px.imshow(vehicles.isnull())

In [ ]:
vehicles.select_dtypes(['Int64','float']).columns

In [ ]:
num_cols=['price', 'year', 'odometer']

In [ ]:
vehicles.select_dtypes(['string']).columns

In [ ]:
obj_cols=['manufacturer', 'model', 'condition', 'cylinders', 'fuel',
       'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color']

### Cleanup & Outlier Analysis

In [ ]:
def remove_NaN_df(df, cols):
    for col in cols:
        df = df[df[col].notna()]
    
    return df

In [ ]:
# removing NaN's from columns that dont carry a significant amount of NaN's or hold values are might skew analysis if 
# improperly guessed, hence will remove those entires ( after careful analysis of the quantity ) to prevent prediction
# errors - e.g. fuel or title_status
cols = ['year','odometer','manufacturer','model','fuel','title_status']
vehicles = remove_NaN_df(vehicles, cols)

In [ ]:
vehicles.info()

In [ ]:
plt.boxplot(data=vehicles, x='price')
plt.show()

In [ ]:
plt.boxplot(data=vehicles, x='year')
plt.show()

In [ ]:
plt.boxplot(data=vehicles, x='odometer')
plt.show()

In [ ]:
vehicles_df = vehicles.copy()

In [ ]:
def find_boundaries(df, variable, distance):
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    lower_boundary = df[variable].quantile(0.25) - (IQR*distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR*distance)
    
    return lower_boundary, upper_boundary

In [ ]:
lo, up = find_boundaries(vehicles_df, 'price', 1.5)
outliers_p = np.where(vehicles_df['price'] > up, True, 
                    np.where(vehicles_df['price'] < lo, True, False))

In [ ]:
vehicles_df=vehicles_df.loc[~outliers_p]

In [ ]:
lo, up = find_boundaries(vehicles_df, 'odometer', 1.5)
outliers_o = np.where(vehicles_df['odometer'] > up, True, 
                    np.where(vehicles_df['odometer'] < lo, True, False))

In [ ]:
vehicles_df=vehicles_df.loc[~outliers_o]

In [ ]:
lo, up = find_boundaries(vehicles_df, 'odometer', 1.5)
outliers_y = np.where(vehicles_df['year'] > up, True, 
                    np.where(vehicles_df['year'] < lo, True, False))

In [ ]:
vehicles_df=vehicles_df.loc[~outliers_y]

In [ ]:
vehicles_df.describe()

In [ ]:
print('% of data removed ===>',((original_row_count-vehicles_df.shape[0])/(original_row_count))*100)

In [ ]:
# remove 'parts only' from the title_status because this category offers no real value  - (NOMIAL datatype)
title_status_values = ['parts only']
vehicles_df = vehicles_df[vehicles_df.title_status.isin(title_status_values) == False]

In [ ]:
# some sanity checks !!!
vehicles_df[vehicles_df['year']==1900]

In [ ]:
plt.boxplot(data=vehicles_df, x='price')
plt.show()

In [ ]:
plt.boxplot(data=vehicles_df, x='year')
plt.show()

In [ ]:
plt.boxplot(data=vehicles_df, x='odometer')
plt.show()

### Visualizations for some interesting price to feature depencies {model, year, odometer, transmission, size, paint_color, type }

In [ ]:
# Summary statistics of the categorical features in the dataset
vehicles_df[obj_cols].describe()

<span style="color:blue">Observation-Summary Statistics</span>

In [ ]:
# create grouped boxplot - which drive is more pricy?
plt.figure(figsize=(20,5))
sns.boxplot(x = vehicles_df['drive'], y = vehicles_df['price'], palette = 'husl')

<span style="color:blue">Observation-price vs year & manufacturer</span>

In [ ]:
fig1 = px.scatter(vehicles_df, x='year', y='price', color='manufacturer')
fig1.show("png")

<span style="color:blue">Observation-price vs odometer</span>

In [ ]:
# which manufacturer is more popular among used car buyers?
plt.figure(figsize=(40,10))
sns.boxplot(x = vehicles_df['manufacturer'], y = vehicles_df['price'], palette = 'husl')

<span style="color:blue">Observation-price vs transmission</span>

In [ ]:
# which model is more expensive
plt.figure(figsize=(20,5))
sns.boxplot(x = vehicles_df['transmission'], y = vehicles_df['price'], palette = 'husl')

<span style="color:blue">Observation-price vs size</span>

In [ ]:
# which size sells the most & how expensive is it
plt.figure(figsize=(20,5))
sns.boxplot(x = vehicles_df['size'], y = vehicles_df['price'], palette = 'husl')

<span style="color:blue">Observation-price vs paint_color</span>

In [ ]:
# Which color is more expensive
plt.figure(figsize=(20,5))
sns.boxplot(x = vehicles_df['paint_color'], y = vehicles_df['price'], palette = 'husl')

<span style="color:blue">Observation-price vs type</span>

### Impute missing categorical values

In [ ]:
# How many NaN's are in each categorical feature
dummy_df = vehicles_df[obj_cols].copy()
dummy_df.isna().sum().reset_index(name="n").plot.bar(x='index', y='n', rot=45)

print(dummy_df.isna().sum().reset_index(name="n"))

In [ ]:
px.imshow(vehicles_df.isnull())

In [ ]:
# Use encoder to encode categorical features.
cols_to_enc = ['manufacturer','model','condition','cylinders','fuel','title_status','transmission','drive','size','type','paint_color']
X = vehicles_df.drop(columns=['price'], axis=1)
y = vehicles_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

encoder = ce.JamesSteinEncoder(cols=cols_to_enc)
X_train_enc = encoder.fit_transform(X_train, y_train)
X_test_enc = encoder.transform(X_test)


In [ ]:
X_train_enc.head()

In [ ]:
X_test_enc.head()

### A Simple Linear Regression - with all features

In [ ]:
%%time
all_features_linreg = ''
linreg_mse = ''

# keeping the intercept term to false
linreg_pipe = Pipeline([('scaler', StandardScaler()), 
                        ('lreg', LinearRegression())]).fit(X_train_enc, y_train)
train_preds = linreg_pipe.predict(X_train_enc)
test_preds = linreg_pipe.predict(X_test_enc)

train_mse = mean_squared_error(y_train, train_preds)
test_mse = mean_squared_error(y_test, test_preds)

print(f'Linear Regression Train MSE: {np.around(train_mse,2)}')
print(f'Linear Regression Test MSE: {np.around(test_mse,2)}')

lr_coef = linreg_pipe.named_steps['lreg'].coef_
lr_intercept = linreg_pipe.named_steps['lreg'].intercept_
print(f'Intercept: {np.around(lr_intercept,2)}')

list_lr_coef = list((zip(linreg_pipe.named_steps['scaler'].get_feature_names_out(), linreg_pipe.named_steps['lreg'].coef_)))
lr_coef_df = pd.DataFrame(list_lr_coef, columns = [' Features', 'Coefficients'])
lr_coef_df.sort_values(by='Coefficients', ascending=False, key=abs)


<span style="color:blue">Observation-Simple Linear Regression</span>

<b>fit_intercept is false: </b>
1. Train MSE: 351884283.44
2. Test MSE: 353533750.09
3. Intercept: 0.0

<b>fit_intercept is True: </b>
1. Train MSE: 66852359.27
2. Test MSE: 71854716.44
3. Intercept: 16882.89


<b>Theory:</b> <i>A positive coefficient indicates that as the value of the independent variable increases, the mean of the dependent variable also tends to increase. A negative coefficient suggests that as the independent variable increases, the dependent variable tends to decrease<i>

At this stage we can draw a quick inference by looking at the coefficients that ones that have a negative affect on the price are <span style="color:red"> odometer, transmission & condition</span>. The more the odometer, the cheaper is the car & so goes with the condition ( old is less expensive ). Model has the most impact on the price followed by the year of the car. Newer makes are more expensive

### Ridge Regression using GridSearchCV

In [ ]:
ridge_pipe = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge())])
param_dict = {'ridge__alpha': [0.001, 0.1, 1.0, 10.0, 100.0, 1000.0]}

In [ ]:
%%time
r_grid = ''
ridge_train_mse = ''
ridge_test_mse = ''
ridge_best_alpha = ''

r_grid = GridSearchCV(ridge_pipe, param_grid=param_dict).fit(X_train_enc, y_train)

train_preds = r_grid.predict(X_train_enc)
test_preds = r_grid.predict(X_test_enc)

ridge_train_mse = mean_squared_error(y_train, train_preds)
ridge_test_mse = mean_squared_error(y_test, test_preds)
ridge_best_alpha = r_grid.best_params_

print(f'Ridge Regression Train MSE: {np.around(ridge_train_mse,2)}')
print(f'Ridge Regression Test MSE: {np.around(ridge_test_mse,2)}')
print(f'Best Alpha: {list(ridge_best_alpha.values())[0]}')

<span style="color:blue">Observation-Ridge Regression</span>

1. Ridge Regression Train MSE: 66852359.42
2. Ridge Regression Test MSE: 71854450.24
3. Best Alpha: 10.0

In [ ]:
ridge_coef_list = []

# for best alpha = 10 find out all the coeffs ( captured in the ridge_best_alpha variable above)
ridge_pipe_4best_alpha = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge(alpha=10))])
ridge_pipe_4best_alpha.fit(X_train_enc, y_train)

ridge_coef_list.append(list(ridge_pipe_4best_alpha.named_steps['ridge'].coef_))
len(ridge_coef_list)
print('For alpha = 10 we have the following coefficients:')
list(zip(X_train_enc.columns, ridge_coef_list[-1]))

ridge_coef_df = pd.DataFrame(list(zip(X_train_enc.columns, ridge_coef_list[-1])), columns = [' Features', 'Coefficients'])
ridge_coef_df.sort_values(by='Coefficients', ascending=False, key=abs)


At this stage, with the best alpha (10), Ridge Regression gives us almost similar results as a simple linear regression. We can draw a quick inference by looking at the coefficients that ones that have a negative affect on the price are <span style="color:red"> odometer, transmission & condition</span>, similar to LR model above. Model & Year have positive affect on the price of the used car vehicle

### LASSO Regression

In [ ]:
### NOTE TO SELF .. make sure you are predicting again with the new pipe

%%time
lasso_grid = ''
lasso_train_mse = ''
lasso_test_mse = ''
lasso_coefs = ''

lasso_pipe = Pipeline([('scaler', StandardScaler()), 
                       ('lasso', Lasso(random_state = 42))]).fit(X_train_enc, y_train)


train_preds = lasso_pipe.predict(X_train_enc)
test_preds = lasso_pipe.predict(X_test_enc)

lasso_train_mse = mean_squared_error(y_train, train_preds)
lasso_test_mse = mean_squared_error(y_test, test_preds)
lasso_coefs = lasso_pipe.named_steps['lasso'].coef_

feature_names = X_train_enc.columns
lasso_df = pd.DataFrame({'feature': feature_names, 'Coefficients': lasso_coefs})

print(f'LASSO Train MSE: {np.around(lasso_train_mse,2)}')
print(f'LASSO Test MSE: {np.around(lasso_test_mse,2)}')

lasso_df.sort_values(by='Coefficients', ascending=False, key=abs)

<span style="color:blue">Observation-LASSO</span>

1. LASSO Train MSE: 69658578.85
2. LASSO Test MSE: 75505613.41

LASSO Regression gives us the same results as the previous 2 regression models with respect to the behvior of the best features with the target

### SFS - To identify a list of features that have the most influence on the price

In [ ]:
sfs_lr_pipe = Pipeline([('scaler', StandardScaler()),
                        ('selector', SequentialFeatureSelector(LinearRegression())),
                        ('lr_model', LinearRegression())])

In [ ]:
%%time
param_dict = {}
sfs_lr_grid = ''
sfs_lr_train_mse = ''
sfs_lr_test_mse = ''

param_dict = {'selector__n_features_to_select': [4, 5, 6]}
sfs_lr_grid = GridSearchCV(sfs_lr_pipe, param_grid=param_dict).fit(X_train_enc, y_train)

train_preds = sfs_lr_grid.predict(X_train_enc)
test_preds = sfs_lr_grid.predict(X_test_enc)

sfs_lr_train_mse = mean_squared_error(y_train, train_preds)
sfs_lr_test_mse = mean_squared_error(y_test, test_preds)

print(f'Minimum Train MSE is : {np.around(sfs_lr_train_mse,2)}')
print(f'Minimum Test MSE is: {np.around(sfs_lr_test_mse,2)}')

In [ ]:
best_estimator = ''
best_selector = ''
best_model = ''
feature_names = ''
coefs = ''

best_estimator = sfs_lr_grid.best_estimator_
best_selector = best_estimator.named_steps['selector']
best_model = sfs_lr_grid.best_estimator_.named_steps['lr_model']
feature_names = X_train_enc.columns[best_selector.get_support()]
coefs = best_model.coef_

print(best_estimator)
print(f'Features from best selector: {feature_names}.')
print('Coefficient values: ')
print('===================')
pd.DataFrame([coefs.T], columns = feature_names, index = ['lr_model'])

### Polynomial Degree & Linear Regression --- To identify the best degree

In [ ]:
%%time
polyd_lr_train_mses = []
polyd_lr_test_mses = []

best_polyd = ''

for i in range(1, 4):
    pipe = Pipeline([('pfeat', PolynomialFeatures(degree = i, include_bias=False)),
                     ('scale', StandardScaler()),
                     ('linreg', LinearRegression())]).fit(X_train_enc, y_train)
    
    train_preds = pipe.predict(X_train_enc)
    test_preds = pipe.predict(X_test_enc)
    polyd_lr_train_mses.append(mean_squared_error(y_train, train_preds))
    polyd_lr_test_mses.append(mean_squared_error(y_test, test_preds))
    
best_polyd_test = polyd_lr_test_mses.index(min(polyd_lr_test_mses)) + 1

print(f'Best TEST performing degree model : {best_polyd_test}')
print(f'Train MSE is: {np.around(polyd_lr_train_mses,2)}')

In [ ]:
print(f'Train MSE is: {np.around(polyd_lr_train_mses,2)}')
print(f'Test MSE is: {np.around(polyd_lr_test_mses,2)}')
best_polyd_train = polyd_lr_train_mses.index(min(polyd_lr_train_mses)) + 1
best_polyd_test = polyd_lr_test_mses.index(min(polyd_lr_test_mses)) + 1

print(f'Best TEST performing degree model : {best_polyd_test}')
print(f'Best TRAIN performing degree model : {best_polyd_train}')

### Polynomial with Degree = 3 & Ridge Regression

In [ ]:
%%time
pd_ridge_pipe = Pipeline([('poly_features', PolynomialFeatures(degree = 3, include_bias= False)),
                          ('scaler', StandardScaler()), 
                          ('ridge', Ridge())])
param_dict = {'ridge__alpha': [0.001, 0.1, 1.0, 10.0, 100.0, 1000.0]}

pd_ridge_grid = ''
pd_ridge_train_mse = ''
pd_ridge_test_mse = ''
pd_ridge_best_alpha = ''

pd_ridge_grid = GridSearchCV(pd_ridge_pipe, param_grid=param_dict).fit(X_train_enc, y_train)

train_preds = pd_ridge_grid.predict(X_train_enc)
test_preds = pd_ridge_grid.predict(X_test_enc)

pd_ridge_train_mse = mean_squared_error(y_train, train_preds)
pd_ridge_test_mse = mean_squared_error(y_test, test_preds)
pd_ridge_best_alpha = pd_ridge_grid.best_params_

print(f'Polynomial with Degree =3 & Ridge Regression Train MSE: {np.around(pd_ridge_train_mse,2)}')
print(f'Polynomial with Degree =3 & Ridge Regression Test MSE: {np.around(pd_ridge_test_mse,2)}')
print(f'Best Alpha: {list(pd_ridge_best_alpha.values())[0]}')

### LASSO Regression with  Degree = 3

In [ ]:
pd_lasso_pipe = Pipeline([('polyfeatures', PolynomialFeatures(degree = 3, include_bias = False)),
                          ('scaler', StandardScaler()),
                          ('lasso', Lasso(random_state = 42))]).fit(X_train_enc, y_train)

train_preds = pd_lasso_pipe.predict(X_train_enc)
test_preds = pd_lasso_pipe.predict(X_test_enc)

lasso_train_mse = mean_squared_error(y_train, train_preds)
lasso_test_mse = mean_squared_error(y_test, test_preds)
lasso_coefs = pd_lasso_pipe.named_steps['lasso'].coef_

pd_lasso_coefs = pd_lasso_pipe.named_steps['lasso'].coef_
feature_names = X_train_enc.columns

print(f'LASSO Train MSE: {np.around(lasso_train_mse,2)}')
print(f'LASSO Test MSE: {np.around(lasso_test_mse,2)}')

list_lasso_coeff = list((zip(pd_lasso_pipe.named_steps['polyfeatures'].get_feature_names_out(), 
                             pd_lasso_pipe.named_steps['lasso'].coef_)))
pd_lasso_df = pd.DataFrame(list_lasso_coeff, columns = [' Features', 'Lasso Coefficients'])
pd_lasso_df.sort_values(by='Lasso Coefficients', ascending=False, key=abs)

In [ ]:
feature_names = pd_lasso_pipe.named_steps['polyfeatures'].get_feature_names_out()
coefs = pd_lasso_pipe.named_steps['lasso'].coef_

print(best_estimator)
print('Coefficient values: ')
print('===================')
errors = pd.DataFrame([coefs.T], columns = feature_names, index = ['lr_model'])
errors[errors.columns[(abs(errors) > 0.000001).any()]]

### Identify the best performing model

In [ ]:
from sklearn.inspection import permutation_importance
pipe = Pipeline([('pfeat', PolynomialFeatures(degree = 3, include_bias=False)),
                     ('scale', StandardScaler()),
                     ('linreg', LinearRegression())]).fit(X_train_enc, y_train)
train_preds = pipe.predict(X_train_enc)
test_preds = pipe.predict(X_test_enc)

from sklearn import metrics
metrics.mean_squared_error(y_test, y_pred, squared = False)

### Permutation Feature Importance with best performing model

In [ ]:
from sklearn.inspection import permutation_importance
r = permutation_importance(pipe, X_test_enc, y_test,
                           random_state=123)
pd.DataFrame({"Variables":X_test_enc.columns,"Score":r.importances_mean}).sort_values(by="Score",ascending = False)

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

# SAMPLE CODE - DO NOT DELETE

In [ ]:
# Generate a heatmap to see which features have a strong corelation with price and use those variables to dig deeper
fig, ax = plt.subplots(figsize=(10,10))
fig.suptitle('Correlation Analysis')
ax = sns.heatmap(vehicles_df.corr(),cmap='RdYlGn', annot=True)